In [7]:
import json
from llm import OpenAI
from agent import Agent1, Agent2

In [2]:
llm_model = OpenAI(api_key="")

In [3]:
with open('config/agent.json') as f1:
    config = json.load(f1)

In [4]:
agent1 = Agent1(llm=llm_model,
                config=config['Agent_1'])
agent2 = Agent2(llm=llm_model,
                config=config['Agent_2'])

In [5]:
query = "Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi, có vợ mới và muốn chia đôi số tài sản, trong đó tiền vốn là của tôi bỏ ra cho mẹ tôi làm ăn. Ông ta đòi làm đơn kiện nếu mẹ tôi không đồng ý chia đôi số tài sản hiện tại. Trường hợp này phải giải quyết như thế nào? (Số tiền tôi đưa mẹ làm ăn không có giấy tờ gì chứng minh cả)"

In [6]:
output_classify = agent1.run(query=query)
if output_classify == 'Có':
    output_analysis = agent2.run(query=query)
    print(output_analysis)
else:
    print("No support")


1. Quy định của pháp luật về việc chia tài sản trong trường hợp sống chung như vợ chồng nhưng không đăng ký kết hôn là gì?
2. Mẹ tôi có quyền yêu cầu chia tài sản trong trường hợp dượng tôi không có quyền sở hữu đối với tài sản đó không?
3. Nếu không có giấy tờ chứng minh số tiền tôi đã đưa cho mẹ làm ăn, tôi có thể bảo vệ quyền lợi của mình như thế nào?
4. Dượng tôi có quyền khởi kiện để yêu cầu chia tài sản trong trường hợp này không?
5. Các bước cần thực hiện để giải quyết tranh chấp tài sản giữa mẹ tôi và dượng tôi là gì?


In [8]:
output_classify = agent1.run(query="Mẹ tôi có quyền gì đối với tài sản mà dượng tôi đã thu lợi từ việc làm ăn mà tôi đã đầu tư không có giấy tờ để chứng minh?")
if output_classify == 'Có':
    output_analysis = agent2.run(query="Mẹ tôi có quyền gì đối với tài sản mà dượng tôi đã thu lợi từ việc làm ăn mà tôi đã đầu tư không có giấy tờ để chứng minh?")
    print(output_analysis)
else:
    print("No support")


1. Mẹ tôi có quyền gì theo luật dân sự đối với tài sản mà dượng tôi đã thu lợi từ việc làm ăn có liên quan đến sự đầu tư của tôi?
2. Việc không có giấy tờ để chứng minh đầu tư có ảnh hưởng gì đến quyền đòi tài sản của mẹ tôi không?
3. Trong trường hợp này, quyền và nghĩa vụ của các bên liên quan được quy định như thế nào trong Bộ luật Dân sự?
4. Có cơ sở pháp lý nào để xác định quyền sở hữu tài sản trong trường hợp không có hợp đồng hay giấy tờ chứng minh không?
5. Nếu xảy ra tranh chấp về tài sản này, mẹ tôi có thể khởi kiện và yêu cầu gì theo quy định của pháp luật?


In [5]:
from knowledge_graph.neo4j_database import Neo4jDatabase

In [6]:
import re

def capture_number(text):
    # Regex pattern to capture the number
    pattern = r"(?<=số\s)\d+"
    # Search for the pattern in the text
    match = re.search(pattern, text)
    number = int(match.group())
    return number

In [7]:
URI = "neo4j://localhost"
AUTH = ("neo4j", "Abc12345")

db = Neo4jDatabase(
        uri=URI, username=AUTH[0], password=AUTH[1]
    )

In [23]:
def get_list_chu_de(db_engine:Neo4jDatabase):
    query = "MATCH (:BoPhapDien)-[r:HAS_SECTION]-(p) RETURN p"

    with db_engine.driver.session() as session:
        result = session.run(query).data()
        result = [ele["p"] for ele in result]
        result = sorted(result, key = lambda x: capture_number(x['title']))
    
    return result
def get_list_de_muc_theo_chu_de_id(db_engine:Neo4jDatabase, chude_id: str):
    query = f"MATCH (a:ChuDePhapdien {{id: \'{chude_id}\'}})-[r:HAS_SECTION]-(p:DeMucPhapdien) RETURN p"

    with db_engine.driver.session() as session:
        result = session.run(query).data()
        result = [ele["p"] for ele in result]
        result = sorted(result, key = lambda x: capture_number(x['title']))
    
    return result

In [24]:
get_list_de_muc_theo_chu_de_id(db_engine = db, chude_id ='efbefb52-bf45-41df-aaf4-6a4e833f333e')

[{'parent': 'efbefb52-bf45-41df-aaf4-6a4e833f333e',
  'id': '23e1929c-6151-4f15-9d67-11d4f7964dc8',
  'label': 'DeMucPhapdien',
  'title': 'Đề mục số 1: Trẻ em',
  'parent_type': 'ChuDePhapdien'},
 {'parent': 'efbefb52-bf45-41df-aaf4-6a4e833f333e',
  'id': '83242e4c-1738-4140-afe5-a5d4f0b8d776',
  'label': 'DeMucPhapdien',
  'title': 'Đề mục số 2: Bình đẳng giới',
  'parent_type': 'ChuDePhapdien'},
 {'parent': 'efbefb52-bf45-41df-aaf4-6a4e833f333e',
  'id': 'c57132ec-b2a3-4e38-9433-afd6cc770a8c',
  'label': 'DeMucPhapdien',
  'title': 'Đề mục số 3: Dân số',
  'parent_type': 'ChuDePhapdien'},
 {'parent': 'efbefb52-bf45-41df-aaf4-6a4e833f333e',
  'id': '4913a1cf-5f78-471c-a807-ed5f8c57aaee',
  'label': 'DeMucPhapdien',
  'title': 'Đề mục số 4: Hôn nhân và gia đình',
  'parent_type': 'ChuDePhapdien'},
 {'parent': 'efbefb52-bf45-41df-aaf4-6a4e833f333e',
  'id': 'dae8ac70-fd7f-41ed-84e8-90980ae92475',
  'label': 'DeMucPhapdien',
  'title': 'Đề mục số 5: Phòng, chống bạo lực gia đình',
  'pa

In [8]:
query = "MATCH (:BoPhapDien)-[r:HAS_SECTION]-(p) RETURN p"

In [9]:
with db.driver.session() as session:
    result = session.run(query).data()
    result = [ele["p"] for ele in result]
    result = sorted(result, key = lambda x: capture_number(x['title']))
result

[{'parent': '00000000000000000000',
  'id': 'c3b69131-2931-4f67-926e-b244e18e8081',
  'label': 'ChuDePhapdien',
  'title': 'Chủ đề số 1: An ninh quốc gia',
  'parent_type': 'BoPhapDien'},
 {'parent': '00000000000000000000',
  'id': '5576c952-dabe-4363-b94a-13419996ff4b',
  'label': 'ChuDePhapdien',
  'title': 'Chủ đề số 2: Bảo hiểm',
  'parent_type': 'BoPhapDien'},
 {'parent': '00000000000000000000',
  'id': 'c124612e-a23f-4199-8747-55fe4e8a8c89',
  'label': 'ChuDePhapdien',
  'title': 'Chủ đề số 3: Bưu chính, viễn thông',
  'parent_type': 'BoPhapDien'},
 {'parent': '00000000000000000000',
  'id': 'e967223c-d26a-4c7f-a8d9-420843bd5bf9',
  'label': 'ChuDePhapdien',
  'title': 'Chủ đề số 4: Bổ trợ tư pháp',
  'parent_type': 'BoPhapDien'},
 {'parent': '00000000000000000000',
  'id': 'cb5c6841-2c38-48ca-b946-da24c8d8a099',
  'label': 'ChuDePhapdien',
  'title': 'Chủ đề số 5: Cán bộ, công chức, viên chức',
  'parent_type': 'BoPhapDien'},
 {'parent': '00000000000000000000',
  'id': 'e4b6a170

In [10]:
num_choices = len(result)

In [28]:
list_chude_content = []
for ix, ele in enumerate(result):
    chude_title = ele['title']
    list_demuc = get_list_de_muc_theo_chu_de_id(db_engine=db, chude_id=ele['id'])
    demuc_content = ', '.join([ele_m['title'] for ele_m in list_demuc])
    chude_content = f"{ix+1}. {chude_title} có các đề mục sau {demuc_content}"
    list_chude_content.append(chude_content)
context_list = "\n\n".join(list_chude_content)

In [30]:
print(context_list)

1. Chủ đề số 1: An ninh quốc gia có các đề mục sau Đề mục số 1: An ninh quốc gia, Đề mục số 2: Bảo vệ bí mật nhà nước, Đề mục số 3: Bảo vệ công trình quan trọng liên quan đến an ninh quốc gia, Đề mục số 4: Biên giới quốc gia, Đề mục số 5: Biển Việt Nam, Đề mục số 6: Công an nhân dân, Đề mục số 7: Cơ yếu, Đề mục số 8: Nhập cảnh, xuất cảnh, quá cảnh, cư trú của người nước ngoài tại Việt Nam, Đề mục số 9: Phòng, chống khủng bố, Đề mục số 11: An ninh mạng, Đề mục số 12: Cảnh vệ

2. Chủ đề số 2: Bảo hiểm có các đề mục sau Đề mục số 1: Bảo hiểm xã hội, Đề mục số 2: Bảo hiểm y tế

3. Chủ đề số 3: Bưu chính, viễn thông có các đề mục sau Đề mục số 1: An toàn thông tin mạng, Đề mục số 2: Bưu chính, Đề mục số 3: Công nghệ thông tin, Đề mục số 5: Tần số vô tuyến điện

4. Chủ đề số 4: Bổ trợ tư pháp có các đề mục sau Đề mục số 1: Đấu giá tài sản, Đề mục số 2: Công chứng, Đề mục số 3: Giám định tư pháp, Đề mục số 4: Luật sư, Đề mục số 5: Trợ giúp pháp lý, Đề mục số 6: Tư vấn pháp luật

5. Chủ đề số 

In [31]:
# context_list = "\n\n".join([f"{ix + 1}. {ele['title']} có các đề mục sau {', '.join([ele_m['title'] for ele_m in get_list_de_muc_theo_chu_de_id(db_engine = db, chude_id =ele[id])])}" for ix, ele in enumerate(result)])

In [50]:
query_str = "Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi, có vợ mới và muốn chia đôi số tài sản, trong đó tiền vốn là của tôi bỏ ra cho mẹ tôi làm ăn. Ông ta đòi làm đơn kiện nếu mẹ tôi không đồng ý chia đôi số tài sản hiện tại. Trường hợp này phải giải quyết như thế nào? (Số tiền tôi đưa mẹ làm ăn không có giấy tờ gì chứng minh cả)"
query_str = "Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay ly hôn thì chia tài sản như thế nào ?"
# query_str = "Tôi biết một người phản quốc. Tôi nên làm gì đây ?"

In [51]:
query_str

'Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay ly hôn thì chia tài sản như thế nào ?'

In [52]:
max_outputs = 3

In [53]:
prompt = """Một số chủ đề pháp luật được đưa ra bên dưới. Chúng được cung cấp trong danh sách được đánh số (1 đến {num_choices}),
---------------------\n
{context_list}
\n---------------------\n
"Chỉ sử dụng các lựa chọn ở trên và không sử dụng kiến ​​thức trước đó, Chỉ chọn chủ đề pháp luật cần thiết, hãy trả về không quá {max_outputs} chủ đề pháp luật có liên quan nhất đến câu hỏi. 
Trả về chỉ số ở đầu mỗi chủ đề theo dạng list. Ví dụ: [2,4,8, 16,32]
Nếu không có chủ đề nào liên quan thì trả về 'Không có chủ đề .'.
Câu hỏi:'{query_str}'\n"""

In [54]:
from llm import OpenAI
llm_model = OpenAI(api_key="",)

In [55]:
select_result_str = llm_model.generate(
    query = prompt.format(
        num_choices=num_choices, 
        context_list=context_list, 
        max_outputs = max_outputs,
        query_str=query_str
    )
)
select_result_str

'[9, 15, 34]'

In [38]:
result[8]

{'parent': '00000000000000000000',
 'id': '8545ecbc-f0ce-44aa-83d6-c43d056f63f2',
 'label': 'ChuDePhapdien',
 'title': 'Chủ đề số 9: Dân sự',
 'parent_type': 'BoPhapDien'}

In [49]:
print(prompt.format(
        num_choices=num_choices, 
        context_list=context_list, 
        max_outputs = max_outputs,
        query_str=query_str
    ))

Một số chủ đề pháp luật được đưa ra bên dưới. Chúng được cung cấp trong danh sách được đánh số (1 đến 43),
---------------------

1. Chủ đề số 1: An ninh quốc gia có các đề mục sau Đề mục số 1: An ninh quốc gia, Đề mục số 2: Bảo vệ bí mật nhà nước, Đề mục số 3: Bảo vệ công trình quan trọng liên quan đến an ninh quốc gia, Đề mục số 4: Biên giới quốc gia, Đề mục số 5: Biển Việt Nam, Đề mục số 6: Công an nhân dân, Đề mục số 7: Cơ yếu, Đề mục số 8: Nhập cảnh, xuất cảnh, quá cảnh, cư trú của người nước ngoài tại Việt Nam, Đề mục số 9: Phòng, chống khủng bố, Đề mục số 11: An ninh mạng, Đề mục số 12: Cảnh vệ

2. Chủ đề số 2: Bảo hiểm có các đề mục sau Đề mục số 1: Bảo hiểm xã hội, Đề mục số 2: Bảo hiểm y tế

3. Chủ đề số 3: Bưu chính, viễn thông có các đề mục sau Đề mục số 1: An toàn thông tin mạng, Đề mục số 2: Bưu chính, Đề mục số 3: Công nghệ thông tin, Đề mục số 5: Tần số vô tuyến điện

4. Chủ đề số 4: Bổ trợ tư pháp có các đề mục sau Đề mục số 1: Đấu giá tài sản, Đề mục số 2: Công chứng,